# Setup

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Instantiate a brand new model

In [2]:
# config file
from config import ModelConfig, TrainConfig
cfg = ModelConfig()
tcfg = TrainConfig()
print(cfg, '\n\n', tcfg)

# import the tokenizer specified by cfg
from tools import import_from_nested_path
imported_objects = import_from_nested_path(['tokenizers', cfg.tokenizer], 'tokenizer', ['get_tokenizer'])
get_tokenizer = imported_objects.get('get_tokenizer')
tokenizer = get_tokenizer(size = cfg.vocab_len)

# model modules
from modules.model import Model
model = Model(cfg).to(cfg.device)

# print the number of parameters in the model
print(f'\n{sum(p.numel() for p in model.parameters())/1e3}K parameters\n')
print(model)

ModelConfig(dim=192, device='cpu', tokenizer='bpe', vocab_len=8192, num_layers=6, second_resid_norm=False, mlp_hidden_mult=4, mlp_bias=False, mlp_nonlinearity='GeLU', mlp_gated=True, num_q_heads=2, num_kv_heads=1, head_dim=96, theta=10000, max_seq_len=512, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06, max_batch_size=1) 

 TrainConfig(weight_decay=0.05, batch_size=24, max_iters=6000, eval_interval=60, eval_samples=1, checkpoint_interval=None, lr_init=1e-06, lr_max=0.1, lr_min=0.001, warmup_iters=60, final_flat_iters=600, anneal_type='cos', num_restarts=3, T_mult=2)

4011.456K parameters

Model(
  (token_embedder): Embedding(8195, 192)
  (layers): ModuleList(
    (0-5): 6 x Layer(
      (pre_attn_norm): Norm()
      (attn): MQA(
        (Wq): Linear(in_features=192, out_features=192, bias=False)
        (Wk): Linear(in_features=192, out_features=96, bias=False)
        (Wv): Linear(in_features=192, out_features=96, bias=False)
        (Wo): Lin

# Training

In [3]:
import torch
from tools import get_data_loader
from train import scheduler_lambda, train

optimizer = torch.optim.AdamW(model.parameters(), lr = tcfg.lr_max, weight_decay = tcfg.weight_decay)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler_lambda)

train_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='train')
test_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='validation')

In [4]:
if False: # set to true if you'd like to see a graph of the learning rate schedule
    import matplotlib.pyplot as plt
    
    # Generate learning rate values
    lrs = [scheduler_lambda(i) for i in range(tcfg.max_iters)]
    
    # Plot the learning rates
    plt.figure(figsize=(10, 5))
    plt.plot(lrs, label='Learning Rate')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Iteration')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.legend()
    plt.show()

In [5]:
model, optimizer, log_data = train(
    model, 
    tokenizer, 
    cfg, 
    optimizer,
    scheduler,
    tcfg, 
    train_data_loader,
    test_data_loader,
    #log_data: list = None, 
    #detect_anomoly = False # use if you're getting crazy errors about a the gradient being broken
)

  0%|                                                                         | 1/6000 [00:07<12:40:26,  7.61s/it]

step 0000: lr 0.000000, train loss 185.6478, val loss 185.1413, ppl inf, time elapsed: 4.47 seconds


  1%|▋                                                                       | 61/6000 [05:35<12:41:10,  7.69s/it]

step 0060: lr 0.010000, train loss 4.7006, val loss 4.5629, ppl 96, time elapsed: 331.78 seconds


  2%|█▍                                                                     | 121/6000 [12:01<12:57:25,  7.93s/it]

...

step 5880: lr 0.000100, train loss 1.1349, val loss 0.9856, ppl 3, time elapsed: 18930.77 seconds


 99%|██████████████████████████████████████████████████████████████████████▎| 5941/6000 [5:18:22<03:26,  3.49s/it]

step 5940: lr 0.000100, train loss 0.9628, val loss 0.9934, ppl 3, time elapsed: 19100.35 seconds


100%|███████████████████████████████████████████████████████████████████████| 6000/6000 [5:21:09<00:00,  3.21s/it]

step 5999: lr 0.000100, train loss 1.0040, val loss 1.0425, ppl 3, time elapsed: 19267.01 seconds


# inference test before you decide to save it

In [6]:
from inference import generate
prompt = "Once upon a time"
model.eval()
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 512,
    temperature = 0.9,
    #memory_saver_div = 8,
    #top_p = 0.9,
    #top_k = 32,
)
model.train()
print(output)

Once upon a time, there was a little girl named Amy. Amy had a ball that she loved to play. She had a toy that her favorite toy loved to play with. One day, she saw a big box. She wanted to find a toy.
Amy asked her mom, "Mom, look at this toy, Mr. Bear is simple." Her mom looked at the toy and said, "You need to fight in the box every day."
Amy put the toy in her toy box and made her strong care of the box. The toy went up and down, and the toy box got driver up. Amy was very happy. She said, "Thank you, Amy! I love you!" She smiled and felt proud that she made a big mess. Amy learned that being gentle is good and kind to others.


# Saving your final model
if `tcfg.checkpoint_interval != None` then checkpoints have already been saved

you DO still need to do this even if you had been saving checkpoints; the final state has not yet been saved

In [7]:
from tools import save_model
save_model(model, cfg, tcfg, log_data)